In [1]:
import random
import time
from deap import base
from deap import creator
from deap import tools
from pyspark import SparkContext

In [2]:
from pyspark import SparkConf

In [3]:
conf = SparkConf().setMaster("local").setAppName("OneMaxDeap")
sc = SparkContext(conf = conf)

In [4]:
# 穿件一个基于RDD的类型

In [5]:
# 创建个体
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax) ## ......

In [7]:
chromoSomeNum = 100 # 一个个体的基因个数
CXPB = 0.5  # 交叉概率
MUTPB = 0.2 # 变异概率
generationNum = 100 # 迭代次数

def evalOneMax(individual):
    return sum(individual) ## ......

toolbox = base.Toolbox()
# 注册个体的特征？？
toolbox.register("attr_bool", random.randint, 0, 1)
# 注册个体
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, chromoSomeNum)
# 注册种群
toolbox.register("population", tools.initRepeat, list, toolbox.individual) ## .......
# 注册评价函数
toolbox.register("evaluate", evalOneMax)
# 注册交叉函数
toolbox.register("mate", tools.cxTwoPoint)
# 注册变异函数
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
# 注册选择函数
toolbox.register("select", tools.selTournament, tournsize=3)



In [8]:
pop = toolbox.population(n=300) ## .........
popRDD = sc.parallelize(pop) ## 转化成RDD
print("Start of evolution")


Start of evolution


In [11]:
type(pop)

list

In [15]:
fitnessRDD = popRDD.map(evalOneMax)
fitValue = fitnessRDD.collect()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 3.0 failed 1 times, most recent failure: Lost task 0.0 in stage 3.0 (TID 3, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/spark-2.2.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 177, in main
    process()
  File "/opt/spark/spark-2.2.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 172, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/opt/spark/spark-2.2.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/serializers.py", line 268, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/opt/spark/spark-2.2.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/serializers.py", line 144, in load_stream
    yield self._read_with_length(stream)
  File

In [14]:
def main():
    random.seed(64) # 随机数种子
    
    pop = toolbox.population(n=300) ## .........
    popRDD = sc.parallelize(pop) ## 转化成RDD
    chromoSomeNum = 100 # 一个个体的基因个数
    CXPB = 0.5  # 交叉概率
    MUTPB = 0.2 # 变异概率
    generationNum = 100 # 迭代次数
    
    print("Start of evolution")
    # 计算种群中每个个体的适应度
    # fitnesses = list(map(toolbox.evaluate, pop))
    fitnessRDD = popRDD.map(toolbox.evaluate)
    # 将计算的适应度分别赋值给种群中的每一个个体，作为其每一个个体的一个属性
    # for ind, fit in zip(pop, fitnesses):
    #     ind.fitness.values = fit
    fitValue = fitnessRDD.collect()
    # popRDD.map()
    
if __name__ == "__main__":
    main() 
    

Start of evolution


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 2.0 failed 1 times, most recent failure: Lost task 0.0 in stage 2.0 (TID 2, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/spark-2.2.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 177, in main
    process()
  File "/opt/spark/spark-2.2.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 172, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/opt/spark/spark-2.2.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/serializers.py", line 268, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/opt/spark/spark-2.2.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/serializers.py", line 144, in load_stream
    yield self._read_with_length(stream)
  File

In [9]:
pop = toolbox.population(n=300)

In [10]:
type(pop)

list